# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Explorng the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [1]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset

In [2]:
# Dataset name
dataset_name = "facebook/ExploreToM"

In [4]:
# TODO: load dataset
ds = load_dataset(dataset_name)

In [9]:
# TODO: number of rows in the dataset
print(ds.shape)

# TODO: Keys in the dataset
print(ds.keys())

# TODO: Feature names
print(ds['train'].features)

# TODO: Display a single row
idx = 5000
for k, v in ds['train'][idx].items():
   print(f'key: {k}\n\t{v}')


{'train': (13309, 18)}
dict_keys(['train'])
{'story_structure': Value('string'), 'infilled_story': Value('string'), 'question': Value('string'), 'expected_answer': Value('string'), 'qprop=params': Value('string'), 'qprop=nth_order': Value('int64'), 'qprop=non_unique_mental_state': Value('bool'), 'sprop=is_false_belief_story_1st': Value('bool'), 'sprop=is_false_belief_story_1st_and_2nd': Value('bool'), 'sprop=story_accuracy_1st_raw': Value('float64'), 'sprop=story_accuracy_1st_infilled': Value('float64'), 'sprop=global_idx': Value('int64'), 'param=story_type': Value('string'), 'param=num_stories_total': Value('int64'), 'param=max_sentences': Value('int64'), 'param=num_people': Value('int64'), 'param=num_moves': Value('int64'), 'param=num_rooms': Value('int64')}
key: story_structure
	Mia entered the hospital staff lounge. Amelia entered the hospital staff lounge. Mia told privately to Madison about the hospital budget cuts. Madison entered the hospital staff lounge. Madison told privatel

In [10]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [16]:
# Initialize the pipeline
qna = pipeline('question-answering', model="distilbert/distilbert-base-cased-distilled-squad")

Device set to use cpu


In [13]:
# TODO: Summarise the text with the pipeline's default model
idx = 10
q = ds['train'][idx]['question']
story = ds['train'][idx]['story_structure']
story = ds['train'][idx]['infilled_story']
a = ds['train'][idx]['expected_answer']

predicted_ans = qna(question=q, context=story)

print(f'predicted: {predicted_ans}')
print(f'actual: {a}')


predicted: {'score': 0.26706773042678833, 'start': 633, 'end': 656, 'answer': 'the ornate wooden chest'}
actual: wooden chest


## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [17]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [18]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [19]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [26]:
# TODO: Encode text
text = "Thou shall not create a machine in the likeness of the human mind."

enc_text = tokenizer(text, return_tensors='pt')

print(enc_text)

input_ids = enc_text.input_ids

print(input_ids)
print(len(input_ids[0]))

dec_text = tokenizer.decode(input_ids[0])
print(dec_text)

for i in input_ids[0]:
   print(f'{i} = {tokenizer.decode(i)}')


{'input_ids': tensor([[  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
          1176,  1757,  1104,  1103,  1769,  1713,   119,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
          1176,  1757,  1104,  1103,  1769,  1713,   119,   102]])
18
[CLS] Thou shall not create a machine in the likeness of the human mind. [SEP]
101 = [CLS]
157 = T
14640 = ##hou
4103 = shall
1136 = not
2561 = create
170 = a
3395 = machine
1107 = in
1103 = the
1176 = like
1757 = ##ness
1104 = of
1103 = the
1769 = human
1713 = mind
119 = .
102 = [SEP]


In [31]:
# TODO: Encoding multiple texts
text0 = "Thou shall not create a machine in the likeness of the human mind."
text1 = "Big black bug bleeds black blood"
texts = [ text0, text1 ]

enc_texts = tokenizer(texts, return_tensors='pt', padding=True)
print(enc_texts)
print(enc_texts.attention_mask)


{'input_ids': tensor([[  101,   157, 14640,  4103,  1136,  2561,   170,  3395,  1107,  1103,
          1176,  1757,  1104,  1103,  1769,  1713,   119,   102],
        [  101,  2562,  1602, 15430, 24752,  1116,  1602,  1892,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [36]:
# TODO: Decode text
dec_text = tokenizer.decode(enc_texts.input_ids[0], skip_special_tokens=True)
print(dec_text)

Thou shall not create a machine in the likeness of the human mind.


## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [ ]:
# TODO: Load libraries
from transformers import AutoModelForQuestionAnswering

# Load the model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [41]:
# TODO: Load question answer model
idx = 20
question = ds['train'][idx]['question']
story = ds['train'][idx]['story_structure']
story = ds['train'][idx]['infilled_story']
answer = ds['train'][idx]['expected_answer']


In [42]:
# TODO: Encode context and question
enc_question = tokenizer(question, return_tensors='pt')
enc_story = tokenizer(story, return_tensors='pt')


In [45]:
# TODO: Tokenize the inputs
inputs = tokenizer(question, story, return_tensors='pt', padding=True)

result = model(inputs.input_ids, inputs.attention_mask)

print(result)
print(result.start_logits)
print(result.end_logits)


QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-1.6762, -5.5590, -5.2944, -5.5207, -7.1013, -6.8303, -6.1847, -6.7702,
         -6.2648, -6.2650, -4.8729, -6.9568, -5.4893, -5.7987, -7.9971, -6.9057,
         -6.8193, -8.1302, -7.2304, -7.7575, -6.5525, -7.1091, -7.9345, -6.2591,
         -5.6751, -8.1623, -5.5561, -5.4626, -2.3985, -2.1461, -1.6427, -4.7434,
         -5.2374, -4.6589, -5.0070, -7.5811, -2.9745, -4.3427, -7.5754, -1.4883,
         -6.6185, -6.5478, -6.1802, -7.7331, -7.8525, -4.6266, -5.8283, -6.6032,
         -6.3370, -5.9505, -4.3827, -4.8499, -7.7499, -6.2904, -8.6796, -6.8582,
         -5.7991, -5.7373, -8.4358, -4.9082, -6.3032, -8.7109, -5.5836, -4.8491,
         -6.4376, -3.3497, -4.8125, -6.8941, -8.4276, -6.2874, -7.3141, -6.9046,
         -2.8717, -7.1186, -7.5646, -6.8215, -6.8792, -7.7840, -6.2740, -6.8292,
         -6.4743, -5.6433, -4.2358, -4.7700, -6.5172, -7.7997, -6.9551, -5.5443,
         -7.4789, -8.0116, -3.3503, -8.5924, -7.1861, -6

In [46]:
import torch
ans_start = torch.argmax(result.start_logits)
ans_end = torch.argmax(result.end_logits) + 1
print(ans_start, ans_end)

tensor(141) tensor(143)


In [48]:
ans_ids = inputs.input_ids[0][ans_start: ans_end]
print(ans_ids)

tensor([22823,  2884])


In [50]:
dec_ans = tokenizer.decode(ans_ids, skip_special_tokens=True)
print(dec_ans)
print(answer)

cardboard box
cardboard box


In [97]:
# Ensure minimum and maximum token length in the answer
def ensure_size(input_ids, answer, min_length = 2, max_length = 5):
   ans_start = torch.argmax(answer['start_logits'])
   ans_end = torch.argmax(answer['end_logits']) + 1
   ans_length = ans_end - ans_start
   if ans_length < min_length:
      ans_end = min(ans_start + min_length, len(input_ids[0]))
   elif ans_length > max_length:
      ans_end = ans_start + max_length
   ans_start = max(0, ans_start)
   ans_end = min(len(input_ids[0]), ans_end)
   return (ans_start, ans_end)

In [ ]:
# TODO Return a minimum of 5 tokens


In [ ]:
# TODO: Try this your self

context = """
Dickens wrote A Christmas Carol during a period when the British were exploring and re-evaluating past Christmas traditions, 
including carols, and newer customs such as cards and Christmas trees. He was influenced by the experiences of his own youth and 
by the Christmas stories of other authors, including Washington Irving and Douglas Jerrold. Dickens had written three Christmas 
stories prior to the novella, and was inspired following a visit to the Field Lane Ragged School, one of several establishments for 
London's street children. The treatment of the poor and the ability of a selfish man to redeem himself by transforming into a more 
sympathetic character are the key themes of the story. There is discussion among academics as to whether this is a fully secular 
story or a Christian allegory.
"""

question = "How many stories has Dickens wrote?"

